# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-05-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-05-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-06 05:22:16,33.93911,67.709953,53105,2244,42666,8195.0,Afghanistan,136.417321,4.225591
1,NaN,NaN,NaN,Albania,2021-01-06 05:22:16,41.15330,20.168300,60283,1210,35551,23522.0,Albania,2094.759886,2.007199
2,NaN,NaN,NaN,Algeria,2021-01-06 05:22:16,28.03390,1.659600,100873,2782,68185,29906.0,Algeria,230.035577,2.757923
3,NaN,NaN,NaN,Andorra,2021-01-06 05:22:16,42.50630,1.521800,8308,84,7585,639.0,Andorra,10752.604672,1.011074
4,NaN,NaN,NaN,Angola,2021-01-06 05:22:16,-11.20270,17.873900,17756,410,11376,5970.0,Angola,54.024996,2.309079


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,51039,51280,51350,51405,51526,51526,51526,51526,53011,53105
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,56254,56572,57146,57727,58316,58316,58991,59438,59623,60283
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,98249,98631,98988,99311,99610,99897,100159,100408,100645,100873


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2160,2174,2179,2181,2191,2191,2191,2191,2237,2244
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1153,1164,1170,1174,1181,1181,1190,1193,1199,1210
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2728,2737,2745,2751,2756,2762,2769,2772,2777,2782


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,41096,41441,41543,41612,41727,41727,41727,41727,42530,42666
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,31565,32122,32700,33185,33634,33634,34353,34648,34996,35551
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,65862,66214,66550,66855,67127,67395,67611,67808,67999,68185


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4029,4065,4105,4164,4190,4239,4268,4305,4336,4546
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,12825,12962,13172,13392,13601,13823,13955,14064,14187,14440
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1406,1417,1462,1492,1514,1517,1528,1530,1533,1575


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,47,47,47,48,48,50,50,50,50,50
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,152,152,156,160,161,169,169,169,169,169
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,32,32,32,32,32,33,33,33,33,33


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
650,1001.0,Autauga,Alabama,US,2021-01-06 05:22:16,32.539527,-86.644082,4546,50,0,4496.0,"Autauga, Alabama, US",8136.891657,1.099868
687,1075.0,Lamar,Alabama,US,2021-01-06 05:22:16,33.779950,-88.096680,1083,15,0,1068.0,"Lamar, Alabama, US",7844.983702,1.385042
688,1077.0,Lauderdale,Alabama,US,2021-01-06 05:22:16,34.901719,-87.656247,6766,70,0,6696.0,"Lauderdale, Alabama, US",7296.530751,1.034585


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,21046195,357258,0
India,10374932,150114,9997272
Brazil,7810400,197732,6994845


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,21046195,357258,0,20688937,2021-01-06 05:22:16,37.936303,-91.379001
India,10374932,150114,9997272,227546,2021-01-06 05:22:16,23.088275,81.806127
Brazil,7810400,197732,6994845,617823,2021-01-06 05:22:16,-12.669522,-48.480493
Russia,3250713,58706,2635242,556765,2021-01-06 05:22:16,54.546312,62.120860
United Kingdom,2782709,76428,6383,2699898,2021-01-06 05:22:16,30.744028,-35.367255


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,2495814,27400,0
Texas,1860763,28797,0
Florida,1392123,22188,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,2495814,27400,0,2468414,2021-01-06 05:22:16,37.843962,-120.728594
Texas,1860763,28797,0,1831966,2021-01-06 05:22:16,31.660643,-98.653069
Florida,1392123,22188,0,1369935,2021-01-06 05:22:16,28.940755,-82.700744
New York,1048281,38743,0,1009538,2021-01-06 05:22:16,42.544151,-75.474183
Illinois,991719,18562,0,973157,2021-01-06 05:22:16,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,840611,11071,0
Illinois,Cook,403987,8523,0
Arizona,Maricopa,349852,5346,0
Florida,Miami-Dade,311606,4257,0
Texas,Harris,249488,3453,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,840611,11071,0,829540,2021-01-06 05:22:16,34.308284,-118.228241,6037.0
Illinois,Cook,403987,8523,0,395464,2021-01-06 05:22:16,41.841448,-87.816588,17031.0
Arizona,Maricopa,349852,5346,0,344506,2021-01-06 05:22:16,33.348359,-112.491815,4013.0
Florida,Miami-Dade,311606,4257,0,307349,2021-01-06 05:22:16,25.611236,-80.551706,12086.0
Texas,Harris,249488,3453,0,246035,2021-01-06 05:22:16,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-03,51526,59438,100408,8192,17642,160,1640718,160027,28504,365768,...,2662699,20823,77258,1,114230,1494,141219,2101,21582,15265
2021-01-04,53011,59623,100645,8249,17684,160,1648940,160220,28517,367410,...,2721622,21426,77258,1,114407,1497,142228,2101,21993,15829
2021-01-05,53105,60283,100873,8308,17756,160,1662730,160544,28536,369721,...,2782709,22104,77295,1,114662,1504,143169,2101,22645,17194


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-03,2191,1193,2772,84,408,5,43482,2850,909,6324,...,75137,204,614,0,1034,35,1470,610,394,380
2021-01-04,2237,1199,2777,84,408,5,43634,2864,909,6357,...,75547,211,614,0,1038,35,1490,610,398,384
2021-01-05,2244,1210,2782,84,410,5,43785,2878,909,6457,...,76428,217,614,0,1042,35,1505,610,412,418


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-03,41727,34648,67808,7517,11223,148,1452960,144091,25778,338831,...,6094,15125,75437,1,108268,1339,121563,1399,19083,11574
2021-01-04,42530,34996,67999,7548,11266,148,1458083,144831,25786,340273,...,6260,15539,75437,1,108468,1339,122976,1401,19229,11716
2021-01-05,42666,35551,68185,7585,11376,148,1466616,145759,25802,343039,...,6383,15919,75545,1,108732,1339,124525,1401,19526,11813


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,4029,4065,4105,4164,4190,4239,4268,4305,4336,4546
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1012,1017,1025,1037,1056,1065,1069,1074,1077,1083
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,6117,6162,6230,6371,6478,6566,6613,6682,6702,6766
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1939,1958,1978,2005,2044,2082,2102,2115,2128,2149
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,9856,9928,10047,10159,10320,10450,10508,10560,10656,10765


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-03,371934,48098,0,556384,233475,2426930,341250,190120,59484,49,...,608297,1810320,283473,7793,2042,363765,255396,91058,526245,44875
2021-01-04,374095,48363,0,561542,234781,2458100,343435,194636,60333,49,...,612250,1829199,285633,7873,2055,367536,256435,91886,527871,45257
2021-01-05,379593,48560,0,567474,238888,2495814,346893,196968,61100,49,...,617649,1860763,288951,8038,2083,371913,258767,93162,531890,45569


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-01-03        4305   14064    1530  1882   4746     900   1546    9731   
2021-01-04        4336   14187    1533  1885   4771     910   1554    9752   
2021-01-05        4546   14440    1575  1923   4849     920   1574    9975   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-03         2402     1438  ...    2081    449     2573      599   
2021-01-04         2415     1441  ...    2114    454     2590      603   
2021-01-05         2474     1454  ...    2136    463     2606      614   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-03           3020  2159  1570          0      783    477  
2021-01-04           3053  2199  1589          0      784    479  
2021-01-05           3061  2216  1597          0      783    479  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-03,4873,215,0,9061,3749,26638,4934,6099,930,0,...,7025,28430,1301,140,23,5124,3459,1376,5261,438
2021-01-04,4878,218,0,9064,3800,26880,4944,6168,937,0,...,7168,28551,1305,144,24,5132,3482,1396,5269,438
2021-01-05,4886,218,0,9317,3836,27400,4991,6192,947,0,...,7267,28797,1312,149,24,5191,3541,1442,5366,464


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-01-03          50     169      33   46     63      22     45     157   
2021-01-04          50     169      33   46     63      22     45     157   
2021-01-05          50     169      33   46     63      22     45     157   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-03           63       22  ...      10      8       17        6   
2021-01-04           63       22  ...      10      8       17        6   
2021-01-05           63       22  ...      11      9       17        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-03             16     4     7          0       19      2  
2021-01-04             16     4     7          0       19      2  
2021-01-05             19     4     7          0       21      3  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-03,0.000000,0.007577,0.002486,0.003184,0.001931,0.006289,0.003599,0.001433,0.000702,0.004024,...,0.021153,0.027028,0.000985,0.0,0.001289,0.008097,0.006644,0.0,0.016580,0.053412
2021-01-04,0.028820,0.003112,0.002360,0.006958,0.002381,0.000000,0.005011,0.001206,0.000456,0.004489,...,0.022129,0.028958,0.000000,0.0,0.001550,0.002008,0.007145,0.0,0.019044,0.036947
2021-01-05,0.001773,0.011070,0.002265,0.007152,0.004071,0.000000,0.008363,0.002022,0.000666,0.006290,...,0.022445,0.031644,0.000479,0.0,0.002229,0.004676,0.006616,0.0,0.029646,0.086234


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-03,0.000000,0.002521,0.001083,0.0,0.002457,0.0,0.002467,0.004937,0.0,0.007809,...,0.006092,0.014925,0.0,NaN,0.001938,0.0,0.016598,0.0,0.005102,0.007958
2021-01-04,0.020995,0.005029,0.001804,0.0,0.000000,0.0,0.003496,0.004912,0.0,0.005218,...,0.005457,0.034314,0.0,NaN,0.003868,0.0,0.013605,0.0,0.010152,0.010526
2021-01-05,0.003129,0.009174,0.001801,0.0,0.004902,0.0,0.003461,0.004888,0.0,0.015731,...,0.011662,0.028436,0.0,NaN,0.003854,0.0,0.010067,0.0,0.035176,0.088542


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-03,0.000000,0.008587,0.002914,0.007236,0.003039,0.0,0.004055,0.003140,0.00000,0.005666,...,0.060933,0.036527,0.002565,0.0,0.001684,0.001496,0.009634,0.002149,0.007444,0.018569
2021-01-04,0.019244,0.010044,0.002817,0.004124,0.003831,0.0,0.003526,0.005136,0.00031,0.004256,...,0.027240,0.027372,0.000000,0.0,0.001847,0.000000,0.011624,0.001430,0.007651,0.012269
2021-01-05,0.003198,0.015859,0.002735,0.004902,0.009764,0.0,0.005852,0.006407,0.00062,0.008129,...,0.019649,0.024455,0.001432,0.0,0.002434,0.000000,0.012596,0.000000,0.015445,0.008279


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-03,0.006702,0.006213,NaN,0.031965,0.008784,0.016065,0.006127,0.000000,0.010378,0.0,...,0.006894,0.009405,0.006458,0.013526,0.000490,0.013965,0.035031,0.019378,0.004952,0.006775
2021-01-04,0.005810,0.005510,NaN,0.009271,0.005594,0.012843,0.006403,0.023753,0.014273,0.0,...,0.006498,0.010429,0.007620,0.010266,0.006366,0.010367,0.004068,0.009093,0.003090,0.008513
2021-01-05,0.014697,0.004073,NaN,0.010564,0.017493,0.015343,0.010069,0.011981,0.012713,0.0,...,0.008818,0.017256,0.011616,0.020958,0.013625,0.011909,0.009094,0.013887,0.007614,0.006894


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-03      0.008669  0.007811  0.001309  0.010199  0.003595  0.008969   
2021-01-04      0.007201  0.008746  0.001961  0.001594  0.005268  0.011111   
2021-01-05      0.048432  0.017833  0.027397  0.020159  0.016349  0.010989   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-03      0.010458  0.004024  0.006706  0.002091  ...  0.022604   
2021-01-04      0.005175  0.002158  0.005412  0.002086  ...  0.015858   
2021-01-05      0.012870  0.022867  0.024431  0.009022  ...  0.010407   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-03      0.018141  0.007045  0.001672   0.009358  0.006527  0.000637   
2021-01-04      0.011136  0.006607  0.006678   0.010927  0.018527  0.012102   
2021-01-05      0.019824  0.006178  0.018242   0.002620  0.007731  0.005035   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-03            NaN  0.001279  0.000000  
2021-01-04            NaN  0.001277  0.004193  
2021-01-05            NaN -0.001276  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-03,0.000205,0.000000,NaN,0.000000,0.005363,0.003617,0.004479,0.000000,0.000000,NaN,...,0.007891,0.003247,0.005410,0.007194,0.000000,0.001368,-0.000578,0.002185,0.000951,0.000000
2021-01-04,0.001026,0.013953,NaN,0.000331,0.013604,0.009085,0.002027,0.011313,0.007527,NaN,...,0.020356,0.004256,0.003075,0.028571,0.043478,0.001561,0.006649,0.014535,0.001521,0.000000
2021-01-05,0.001640,0.000000,NaN,0.027913,0.009474,0.019345,0.009506,0.003891,0.010672,NaN,...,0.013811,0.008616,0.005364,0.034722,0.000000,0.011496,0.016944,0.032951,0.018410,0.059361


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-01-03         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2021-01-04         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   
2021-01-05         0.0     0.0     0.0  0.0    0.0     0.0    0.0     0.0   

Province_State                    ... Wyoming                            \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan  Sublette   
2021-01-03          0.0      0.0  ...     0.0  0.000      0.0  0.000000   
2021-01-04          0.0      0.0  ...     0.0  0.000      0.0  0.000000   
2021-01-05          0.0      0.0  ...     0.1  0.125      0.0  0.166667   

Province_State                                                     
Admin2         Sweetwater Teton Uinta Unassigned  Washakie Weston  
2021-01-03         0.0000   0.0   0.0        NaN  0.000000    0.0  
2021-01-04         0.0000   0.0   0.0        NaN  0.000000    0.0  
2021-01-05         0.1875   0.0   0.0        NaN  0.105263    0.5  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-03,0.000261,0.007907,0.002669,0.005099,0.002313,0.003700,0.003772,0.001460,0.000813,0.004583,...,0.021634,0.028430,0.000816,0.0,0.001831,0.006703,0.007495,0.000206,0.014242,0.038017
2021-01-04,0.014541,0.005510,0.002515,0.006029,0.002347,0.001850,0.004391,0.001333,0.000635,0.004536,...,0.021882,0.028694,0.000408,0.0,0.001690,0.004355,0.007320,0.000103,0.016643,0.037482
2021-01-05,0.008157,0.008290,0.002390,0.006590,0.003209,0.000925,0.006377,0.001678,0.000650,0.005413,...,0.022163,0.030169,0.000444,0.0,0.001960,0.004516,0.006968,0.000051,0.023144,0.061858


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-03,0.000443,0.003865,0.001721,0.000196,0.002720,5.960464e-08,0.002110,0.004059,0.000004,0.006927,...,0.007029,0.031075,0.000057,NaN,0.002098,3.739979e-39,0.017137,0.000084,0.004962,0.012521
2021-01-04,0.010719,0.004447,0.001762,0.000098,0.001360,2.980232e-08,0.002803,0.004486,0.000002,0.006072,...,0.006243,0.032694,0.000029,NaN,0.002983,1.869990e-39,0.015371,0.000042,0.007557,0.011524
2021-01-05,0.006924,0.006811,0.001781,0.000049,0.003131,1.490116e-08,0.003132,0.004687,0.000001,0.010902,...,0.008952,0.030565,0.000014,NaN,0.003418,9.349948e-40,0.012719,0.000021,0.021367,0.050033


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-03,0.000387,0.011472,0.003319,0.004813,0.005977,0.000271,0.005881,0.003387,0.000136,0.005892,...,0.040162,0.038823,0.002050,0.0,0.002010,0.003411,0.011981,0.00149,0.007550,0.012167
2021-01-04,0.009816,0.010758,0.003068,0.004469,0.004904,0.000135,0.004704,0.004261,0.000223,0.005074,...,0.033701,0.033097,0.001025,0.0,0.001929,0.001706,0.011802,0.00146,0.007600,0.012218
2021-01-05,0.006507,0.013309,0.002902,0.004685,0.007334,0.000068,0.005278,0.005334,0.000422,0.006601,...,0.026675,0.028776,0.001228,0.0,0.002181,0.000853,0.012199,0.00073,0.011523,0.010249


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-03,0.008950,0.008518,NaN,0.024162,0.010478,0.017926,0.006711,0.007387,0.011526,2.139111e-89,...,0.012192,0.008914,0.009422,0.017978,0.002323,0.013305,0.019096,0.019415,0.004365,0.005051
2021-01-04,0.007380,0.007014,NaN,0.016716,0.008036,0.015385,0.006557,0.015570,0.012899,1.069556e-89,...,0.009345,0.009671,0.008521,0.014122,0.004344,0.011836,0.011582,0.014254,0.003727,0.006782
2021-01-05,0.011038,0.005543,NaN,0.013640,0.012764,0.015364,0.008313,0.013776,0.012806,5.347778e-90,...,0.009082,0.013463,0.010069,0.017540,0.008985,0.011872,0.010338,0.014070,0.005670,0.006838


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-03      0.008628  0.010235  0.004873  0.009089  0.006481  0.011489   
2021-01-04      0.007915  0.009491  0.003417  0.005342  0.005874  0.011300   
2021-01-05      0.028173  0.013662  0.015407  0.012750  0.011111  0.011145   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-03      0.008888  0.006918  0.008529  0.004423  ...  0.014860   
2021-01-04      0.007031  0.004538  0.006971  0.003255  ...  0.015359   
2021-01-05      0.009951  0.013702  0.015701  0.006138  ...  0.012883   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-03      0.011344  0.005642  0.002063   0.007770  0.004893  0.003439   
2021-01-04      0.011240  0.006124  0.004370   0.009349  0.011710  0.007771   
2021-01-05      0.015532  0.006151  0.011306   0.005985  0.009720  0.006403   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-03           -1.0  0.002420  0.001125  
2021-01-04           -1.0  0.001849  0.002659  
2021-01-05           -1.0  0.000287  0.001329  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-03,0.002351,0.012260,NaN,0.005118,0.006533,0.008443,0.007300,0.005086,0.001719,NaN,...,0.008020,0.005088,0.009075,0.011486,3.003852e-20,0.004915,0.001818,0.007589,0.002043,0.005771
2021-01-04,0.001688,0.013107,NaN,0.002725,0.010068,0.008764,0.004663,0.008200,0.004623,NaN,...,0.014188,0.004672,0.006075,0.020029,2.173913e-02,0.003238,0.004234,0.011062,0.001782,0.002885
2021-01-05,0.001664,0.006553,NaN,0.015319,0.009771,0.014055,0.007085,0.006045,0.007648,NaN,...,0.014000,0.006644,0.005719,0.027376,1.086957e-02,0.007367,0.010589,0.022007,0.010096,0.031123


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-03      0.005969  0.007845  0.003907  0.002976  0.002717  0.006250   
2021-01-04      0.002985  0.003923  0.001954  0.001488  0.001358  0.003125   
2021-01-05      0.001492  0.001961  0.000977  0.000744  0.000679  0.001563   

Province_State                                              ...   Wyoming  \
Admin2                Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-03      1.762340e-07  0.002488  0.001523 -0.001359  ...  0.026786   
2021-01-04      8.811700e-08  0.001244  0.000761 -0.000680  ...  0.013393   
2021-01-05      4.405850e-08  0.000622  0.000381 -0.000340  ...  0.056696   

Province_State                                                         \
Admin2                Platte  Sheridan  Sublette Sweetwater     Teton   
2021-01-03      8.512549e-12  0.004428  0.012500   0.004167  0.024740   
2021-01-04      4.256274e-12  0.002214  0.006250   0.002083  0.012370   
2021-01-05      6.250000e-02  0.001107  0.086458   0.094792  0.006185   

Province_State                                                   
Admin2                 Uinta Unassigned  Washakie        Weston  
2021-01-03      3.286793e-09  -0.501961  0.014334  5.820765e-11  
2021-01-04      1.643396e-09  -0.501961  0.007167  2.910383e-11  
2021-01-05      8.216981e-10  -0.501961  0.056215  2.500000e-01  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210105,AK,47006,NaN,1255853,NaN,totalTestsViral,1302859,93.0,1056.0,...,1302859,0,23,d12726eb32f967aa370925e4ec95bebabe3b07a0,0,0,0,0,0,NaN
1,20210105,AL,379593,74503.0,1605791,NaN,totalTestsPeopleViral,1910881,3080.0,35430.0,...,1985384,8,0,a4321a0590c59e924d2aaefc9fdb48de3dbc7822,0,0,0,0,0,NaN
2,20210105,AR,238888,42958.0,1908393,NaN,totalTestsViral,2104323,1323.0,11743.0,...,2147281,36,229,ffc1b0d3bf31e09303f57be99d2814c8f34fb6cf,0,0,0,0,0,NaN
3,20210105,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,6fbae5d9894b29faae54e071b0884b594f8738e2,0,0,0,0,0,NaN
4,20210105,AZ,567474,28755.0,2415013,NaN,totalTestsViral,5324030,4789.0,40303.0,...,2982487,253,406,d9299956304255d01cca5c425249c05d86ba33bc,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

/Users/Brian/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210105,AK,47006.0,NaN,1255853.0,NaN,totalTestsViral,1302859.0,93.0,1056.0,...,1302859,0,23,d12726eb32f967aa370925e4ec95bebabe3b07a0,0,0,0,0,0,NaN
1,20210105,AL,379593.0,74503.0,1605791.0,NaN,totalTestsPeopleViral,1910881.0,3080.0,35430.0,...,1985384,8,0,a4321a0590c59e924d2aaefc9fdb48de3dbc7822,0,0,0,0,0,NaN
2,20210105,AR,238888.0,42958.0,1908393.0,NaN,totalTestsViral,2104323.0,1323.0,11743.0,...,2147281,36,229,ffc1b0d3bf31e09303f57be99d2814c8f34fb6cf,0,0,0,0,0,NaN
3,20210105,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,6fbae5d9894b29faae54e071b0884b594f8738e2,0,0,0,0,0,NaN
4,20210105,AZ,567474.0,28755.0,2415013.0,NaN,totalTestsViral,5324030.0,4789.0,40303.0,...,2982487,253,406,d9299956304255d01cca5c425249c05d86ba33bc,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-05,AK,47006,NaN,1255853,NaN,totalTestsViral,1302859,93.0,1056.0,NaN,...,1302859,0,23,d12726eb32f967aa370925e4ec95bebabe3b07a0,0,0,0,0,0,NaN
2021-01-05,AL,379593,74503.0,1605791,NaN,totalTestsPeopleViral,1910881,3080.0,35430.0,NaN,...,1985384,8,0,a4321a0590c59e924d2aaefc9fdb48de3dbc7822,0,0,0,0,0,NaN
2021-01-05,AR,238888,42958.0,1908393,NaN,totalTestsViral,2104323,1323.0,11743.0,426.0,...,2147281,36,229,ffc1b0d3bf31e09303f57be99d2814c8f34fb6cf,0,0,0,0,0,NaN
2021-01-05,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,6fbae5d9894b29faae54e071b0884b594f8738e2,0,0,0,0,0,NaN
2021-01-05,AZ,567474,28755.0,2415013,NaN,totalTestsViral,5324030,4789.0,40303.0,1096.0,...,2982487,253,406,d9299956304255d01cca5c425249c05d86ba33bc,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-05,AK,47006.0,NaN,1255853.0,NaN,totalTestsViral,1302859.0,93.0,1056.0,NaN,...,1302859,0,23,d12726eb32f967aa370925e4ec95bebabe3b07a0,0,0,0,0,0,NaN
2021-01-05,AL,379593.0,74503.0,1605791.0,NaN,totalTestsPeopleViral,1910881.0,3080.0,35430.0,NaN,...,1985384,8,0,a4321a0590c59e924d2aaefc9fdb48de3dbc7822,0,0,0,0,0,NaN
2021-01-05,AR,238888.0,42958.0,1908393.0,NaN,totalTestsViral,2104323.0,1323.0,11743.0,426.0,...,2147281,36,229,ffc1b0d3bf31e09303f57be99d2814c8f34fb6cf,0,0,0,0,0,NaN
2021-01-05,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,6fbae5d9894b29faae54e071b0884b594f8738e2,0,0,0,0,0,NaN
2021-01-05,AZ,567474.0,28755.0,2415013.0,NaN,totalTestsViral,5324030.0,4789.0,40303.0,1096.0,...,2982487,253,406,d9299956304255d01cca5c425249c05d86ba33bc,0,0,0,0,0,NaN
